In [1]:
# Detecting Objects on Video with OpenCV deep learning library
#
# Algorithm:
# Reading input video --> Loading YOLO v3 Network -->
# --> Reading frames in the loop --> Getting blob from the frame -->
# --> Implementing Forward Pass --> Getting Bounding Boxes -->
# --> Non-maximum Suppression --> Drawing Bounding Boxes with Labels -->
# --> Writing processed frames
#
# Result:
# New video file with Detected Objects, Bounding Boxes and Labels


# Importing needed libraries
import numpy as np
import cv2
import time


# Keywords extraction

from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import en_core_web_lg
nlp = en_core_web_lg.load()



import nltk
from pprint import pprint
import pandas as pd
from nltk.tokenize import sent_tokenize


class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):

        keyword_key = ""
    

        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            
            keyword_key = str(keyword_key) + "," + str(key)
            if i > number:
                break
        
        print(keyword_key)
        return keyword_key
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight




def keywordsExtraction(text) :

    tr4w = TextRank4Keyword()
    tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
    
    keyword_key = ""

    """Print top number keywords"""
    node_weight = OrderedDict(sorted(tr4w.node_weight.items(), key=lambda t: t[1], reverse=True))
    list_keyword_key = []
    for i, (key, value) in enumerate(node_weight.items()):
        #print(key + ' - ' + str(value))
        
        # keyword_key = str(keyword_key) + "," + str(key)
        list_keyword_key.append(str(key))
        
        if i > 30:
            break
    
    
    
    #print(list_keyword_key)
    return list_keyword_key


# input scenairo! >>> 외부 문서에서 불러올 것!
# scenairo = """ """
scenairo = open('script/Script.txt', 'r')
scenairo = scenairo.read()

# keywords Extract! 리스트 형태로 출력, 각 값을 이미지 인식 레이블과 비교할 것, 이 결과를 가지로 인식되는 이미지의 레이블과 비교해도 되고 아래의 명사추출 결과만을 가지고 비교해도 됨
keywordsExtraction(scenairo)

print("keyword extraction:", keywordsExtraction(scenairo))




# 사니리오에서 주요 명사만 추출한다면,
def build_dictionary(text_):
    
    essay_input_corpus = str(text_) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    dictionary = {}
    for sent in sentences:
        pos_tags = nltk.pos_tag(nltk.word_tokenize(sent))
        for tag in pos_tags:
            value = tag[0]
            pos = tag[1]
            dictionary[value] = pos
    return dictionary

pos_dict = build_dictionary(scenairo)

nouns = [n for n, tag in pos_dict.items() if tag in ["NN","NNP"] ]
print("시나리오에서 주요명사 추출",nouns) # 이 결과값(리스트 형식으로 저장됨)을 이미지 인식 레이블과 비교하여, 인식되는 프레임의 시간값을 추출하여 별도 저장하여 편집구간을 찾아낼 것! 


keyword extraction: ['Donny', 'video', 'Anthony', 'past', 'bear', 'time', 'apartment', 'footage', 'film', 'life', 'friend', 'present', 'Scene', 'reaction', 'thing', 'couple', 'Flashback', 'bus', 'Yo', 'media', 'videos', 'notes', 'V', 'log', 'date', 'woods', 'look', 'living', 'delivery', 'frustration', 'failures', 'Untitled']
시나리오에서 주요명사 추출 ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', '

In [2]:
"""
Start of:
Reading input video
"""

# Defining 'VideoCapture' object
# and reading video from a file
# Pay attention! If you're using Windows, the path might looks like:
# r'videos\traffic-cars.mp4'
# or:
# 'videos\\traffic-cars.mp4'
video = cv2.VideoCapture('/Users/kimkwangil/Documents/VISION2020/김윤하_영상AI/movie/pre_movie_clip.mp4')

# Preparing variable for writer
# that we will use to write processed frames
writer = None

# Preparing variables for spatial dimensions of the frames
h, w = None, None

length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print("video length:", length)
fps_ = video.get(cv2.CAP_PROP_FPS)
print("fps_:", fps_)
duration = length / fps_
print("duration:", duration)
minutes = int(duration / 60)
print("minutes:", minutes)
seconds = duration % 60
print("seconds:", seconds)



video length: 135977
fps_: 59.94005994005994
duration: 2268.5496166666667
minutes: 37
seconds: 48.54961666666668


In [3]:
with open('yolo-coco-data/coco.names') as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# Check point
print('List with labels names:')
print(labels)

List with labels names:
['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()
# print("layers_names_all", layers_names_all)
# Check point
print()
print(layers_names_all)


['conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'leaky

In [5]:
# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

# Check point
print()
print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']


['yolo_82', 'yolo_94', 'yolo_106']


In [6]:
# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Check point
print()
print(type(colours))  # <class 'numpy.ndarray'>
print(colours.shape)  # (80, 3)
print(colours[0])  # [172  10 127]


<class 'numpy.ndarray'>
(80, 3)
[124  61 103]


In [7]:

"""
Start of:
Reading frames in the loop
"""

# Defining variable for counting frames
# At the end we will show total amount of processed frames
f = 0

# Defining variable for counting total time
# At the end we will show time spent for processing all frames
t = 0

#previous time 설정
prev_time = 0

# Defining loop for catching frames
recorded_time = []
#동영상 플레이 시간 측정위해서 시작되는 시간 추출(컴퓨터의 시간)
prev_time = time.time()
    
while True:
    # 동영상을 읽어준다. 영상을 모두 불러오기 위해서 while 문 사용
    ret, frame = video.read()  # 프레임을 읽어오면 ret = True, list 형식으로 frame 추출

    # 컴퓨터의 시간값 가져오기

    
    currnet_time = time.time() 
    # 프레임의 시간은 컴퓨터의 시간에서 이전 프레임 시간을 빼면 됨
    frame_time = currnet_time - prev_time
    #print("frame_time:", frame_time) # >>>> current_time: 1610578028.219424
    recorded_time.append(frame_time) # 시간 기록

    # If the frame was not retrieved
    # e.g.: at the end of the video,
    # then we break the loop
    if not ret: 
        break

    # Getting spatial dimensions of the frame
    # we do it only once from the very beginning
    # all other frames have the same dimension
    if w is None or h is None:
        # Slicing from tuple only first two elements
        h, w = frame.shape[:2]

    """
    Start of:
    Getting blob from current frame
    """

    # Getting blob from current frame
    # The 'cv2.dnn.blobFromImage' function returns 4-dimensional blob from current
    # frame after mean subtraction, normalizing, and RB channels swapping
    # Resulted shape has number of frames, number of channels, width and height
    # E.G.:
    # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)

    """
    End of:
    Getting blob from current frame
    """

    """
    Start of:
    Implementing Forward pass
    """

    # Implementing forward pass with our blob and only through output layers
    # Calculating at the same time, needed time for forward pass
    network.setInput(blob)  # setting blob as input to the network
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    # print("output_from_network:", output_from_network)
    end = time.time()

    # Increasing counters for frames and total time
    f += 1
    t += end - start

    # Showing spent time for single current frame
    # print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

    """
    End of:
    Implementing Forward pass
    """

    """
    Start of:
    Getting bounding boxes
    """

    # Preparing lists for detected bounding boxes,
    # obtained confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []
    
    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # # Check point
            # # Every 'detected_objects' numpy array has first 4 numbers with
            # # bounding box coordinates and rest 80 with probabilities
            #  # for every class
            # print(detected_objects.shape)  # (85,)

            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                # YOLO data format keeps coordinates for center of bounding box
                # and its current width and height
                # That is why we can just multiply them elementwise
                # to the width and height
                # of the original frame and in this way get coordinates for center
                # of bounding box, its width and height for original frame
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Now, from YOLO data format, we can get top left corner coordinates
                # that are x_min and y_min
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min,
                                       int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    # 연산량 줄이기, 바운딩 박스가 여러개 그려지는 것을 방지
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold)
    
    """
    Start of:
    Drawing bounding boxes and labels
    """

    # Checking if there is at least one detected object
    # after non-maximum suppression
    if len(results) > 0: # if openCV get detected imgs
        # Going through indexes of results
        for i in results.flatten():
            # Getting current bounding box coordinates,
            # its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Preparing colour for current bounding box
            # and converting from numpy array to list
            colour_box_current = colours[class_numbers[i]].tolist()

            # # # Check point
            # print(type(colour_box_current))  # <class 'list'>
            # print(colour_box_current)  # [172 , 10, 127]

            # Drawing bounding box on the original current frame
            cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)



            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])], # >>>>>>>>>>> 여기의 레이블과 사니라오의 토픽이 같으면, 시간을 기록하거나 화면에 편집구간 표시할 것
                                                   confidences[i])
            print("text_box_current_인식결과:", labels[int(class_numbers[i])])

            label_name = labels[int(class_numbers[i])] #save detected label of img

            # print("nouns type :", type(nouns))
            # print("label_name type :", type(label_name))
            print("nouns:", nouns)
            print("label_name:", label_name)

            #####################################################   
            
            # label_name 과 추출한 nouns 값이 같으면, 그 프레임의 시간을 txt파일로 저장한다.
            
            editguide_dict = {} #영상에서 인식한 키워드(레이블 값)과 인식된 시간값을 딕셔너리로 저장한다. 이것을 문서로 최종 저장하면 됨
            for noun_ in nouns:
                if noun_ == label_name:
                    print("이미지기 인식된 순간 프레임 타임 기록:",frame_time )
                    print("스크립트에서 추출한 단어", nouns)
                    recorded_time.append(frame_time) # 시간 기록
                    editguide_dict[label_name] = frame_time

            # Putting text with label and confidence on the original image
            cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    """
    End of:
    Drawing bounding boxes and labels
    """
    
    """
    Start of:
    Writing processed frame into the file
    """

    # Initializing writer
    # we do it only once from the very beginning
    # when we get spatial dimensions of the frames
    if writer is None:
        # Constructing code of the codec
        # to be used in the function VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        # Pay attention! If you're using Windows, yours path might looks like:
        # r'videos\result-traffic-cars.mp4'
        # or:
        # 'videos\\result-traffic-cars.mp4'
        writer = cv2.VideoWriter('result_video/result-Scence1_0319.mp4', fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)

    """
    End of:
    Writing processed frame into the file
    """

"""
End of:
Reading frames in the loop
"""


text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

...

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

text_box_current_인식결과: person
nouns: ['\ufeffrevision', 'title', 'starting', 'donny', 'anthony', 'director', 'focus', '’', 'trial', 'process', 'reaction', 'scene', 'home/flashback', 'bus', 'apartment', 'phone', 'date', 'day', ']', 'look', 'frustration', 'flashback', 'message', 'friend', 'background', 't', 'way', 'mother', 'back', 'home', 'morality', 'life', 'family', 'pleasure', 'cool', 'deviator', 've', 'couple', 'rent', 'thing', 'living', 'm', 'mess', 'window', 'backpack', 'notepad', 'end', 'upload', 'note', 'idea', 'structure', 'time', 'audience', '“', '”', 'video', 'beginning', 'film', 'movie', 'videos', 'cycle', 'huh', 'camera', 'today', 'bear', 'footage', 'room', 'cut', 'style', 'mama', 'papa', 'line', 'delivery', 'holy', 'cub', 'forehead', 'oh', 'face', 'palms', 'god', 'please', 'soul', 'laughter', 'desk', 'anger', 'insane', 'keyboard', 'm…', 'man', 'ground', 'hey', 'yeah', 'world', 'counting', 'money', 'fast', 'job', 'relax', 'act', 'formula', 'clothing', 'course', 'something',

KeyboardInterrupt: 

In [8]:
# 플레이 시간 기록 잘됨
recorded_time

[0.007219076156616211,
 0.38915586471557617,
 0.6234681606292725,
 0.9699490070343018,
 1.2066099643707275,
 1.4434340000152588,
 1.7797880172729492,
 2.0107719898223877,
 2.2550580501556396,
 2.498136043548584,
 2.74790096282959,
 2.9973771572113037,
 3.2883028984069824,
 3.5262808799743652,
 3.769784927368164,
 4.0141520500183105,
 4.249176979064941,
 4.473942995071411,
 4.713681936264038,
 4.953743934631348,
 5.197249174118042,
 5.427897930145264,
 5.664413928985596,
 5.906031847000122,
 6.136765003204346,
 6.375869035720825,
 6.6047680377960205,
 6.851491928100586,
 7.122361898422241,
 7.373053073883057,
 7.61471700668335,
 7.848056077957153,
 8.082328796386719,
 8.3684401512146,
 8.692984104156494,
 8.943667888641357,
 9.183480024337769,
 9.419399976730347,
 9.648918151855469,
 9.943642854690552,
 10.203943014144897,
 10.430423021316528,
 10.66641902923584,
 10.91067099571228,
 11.157472133636475,
 11.383817911148071,
 11.680181980133057,
 11.952163934707642,
 12.180393934249878,


In [9]:
# Printing final results
print()
# print('Total number of frames', f)
# print('Total amount of time {:.5f} seconds'.format(t))
# print('FPS:', round((f / t), 1))
print('recorded_time:', recorded_time) # 시나리오에서 추출한 키워드, 명사 와 같은 오브젝트가 인식되면, 인식된 시간을 기록한다.
print('editguide_dict', editguide_dict) # 시나리오기반 키워드로 인식한 오브젝트의 추출 시간을 딕셔너리 형식으로 기록
# 이 시간기록을 가지로 편집할 때 활용한다.


# Releasing video reader and writer
video.release()
writer.release()


recorded_time: [0.007219076156616211, 0.38915586471557617, 0.6234681606292725, 0.9699490070343018, 1.2066099643707275, 1.4434340000152588, 1.7797880172729492, 2.0107719898223877, 2.2550580501556396, 2.498136043548584, 2.74790096282959, 2.9973771572113037, 3.2883028984069824, 3.5262808799743652, 3.769784927368164, 4.0141520500183105, 4.249176979064941, 4.473942995071411, 4.713681936264038, 4.953743934631348, 5.197249174118042, 5.427897930145264, 5.664413928985596, 5.906031847000122, 6.136765003204346, 6.375869035720825, 6.6047680377960205, 6.851491928100586, 7.122361898422241, 7.373053073883057, 7.61471700668335, 7.848056077957153, 8.082328796386719, 8.3684401512146, 8.692984104156494, 8.943667888641357, 9.183480024337769, 9.419399976730347, 9.648918151855469, 9.943642854690552, 10.203943014144897, 10.430423021316528, 10.66641902923584, 10.91067099571228, 11.157472133636475, 11.383817911148071, 11.680181980133057, 11.952163934707642, 12.180393934249878, 12.40605902671814, 12.6417930126